In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
essalud = pd.ExcelFile('data/EsSalud en Cifras_Preliminar Julio  2018.xlsx')

In [5]:
sheet_names = essalud.sheet_names[1:-8]

In [6]:
sheet_names.index('Psicoprofilaxis de Enfermería')

76

In [7]:
print('Eliminado por no aparecer en base 2017: ', sheet_names[41], ',', sheet_names[42])
print('Eliminado por no aparecer en base 2018: ', sheet_names[62], ',', sheet_names[70], ',', sheet_names[76])

Eliminado por no aparecer en base 2017:  Prioridad V , N° Camillas de Emergencia
Eliminado por no aparecer en base 2018:  Primera Consulta Médico Prev , Psicoprofilaxis de Nutrición , Psicoprofilaxis de Enfermería


In [8]:
sheet_names.pop(41);
sheet_names.pop(42-1);
sheet_names.pop(62-2);
sheet_names.pop(70-3);
sheet_names.pop(76-4);

In [9]:
dfs = [pd.DataFrame(columns=['nivel-centro', 'periodo']+[sheet_name]) for sheet_name in sheet_names]

n_sheet = 0
for sheet_name in tqdm(sheet_names):
    df = essalud.parse(sheet_name=sheet_name, skiprows=13)
    df = df[df['Red'].isna()].drop(labels=['Red','Suma de Total'], axis=1)
    df['nivel_centro'] = df['nivel'] + '-' + df['Centro']
    df.index = df['nivel_centro']
    
    df_temp = df.iloc[:,2:-1]
    periods_names = ['per_'+ elem[-6:-2] + '-' + elem[-2:] for elem in df.columns[2:-1]]
    df_temp.columns = periods_names
    
    ix = 0
    for row in df_temp.itertuples():
        hospital = row.Index
        for period_n, value in enumerate(row[1:]):
            dfs[n_sheet].loc[ix,'nivel-centro'] = hospital
            dfs[n_sheet].loc[ix,'periodo'] = periods_names[period_n]
            dfs[n_sheet].loc[ix,sheet_name] = value
            ix += 1
    n_sheet += 1

100%|██████████| 75/75 [03:01<00:00,  1.60s/it]


In [14]:
for i in range(len(dfs)):
    dfs[i].index = dfs[i]['nivel-centro'] + '_' + dfs[i]['periodo']
    dfs[i] = dfs[i].drop(labels=['nivel-centro','periodo'], axis=1)

In [15]:
len(dfs)

75

In [17]:
final_df = dfs[0]

for i in range(1,len(dfs)):
    final_df = final_df.join(dfs[i], how='outer')

In [19]:
final_df.to_csv('data/essalud_data_2018.csv')